In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Set style for better visualizations
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Configure pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

print("Libraries imported successfully!")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")


In [ ]:
# Column mapping as provided in the assignment
column_mapping = {
    'Timestamp': 'Timestamp',
    'Oxygen enrichment rate': 'OxEnRa',
    'Blast furnace permeability index': 'BlFuPeIn',
    'Enriching oxygen flow': 'EnOxFl',
    'Cold blast flow': 'CoBlFl',
    'Blast momentum': 'BlMo',
    'Blast furnace bosh gas volume': 'BlFuBoGaVo',
    'Blast furnace bosh gas index': 'BlFuBoGaIn',
    'Theoretical combustion temperature': 'ThCoTe',
    'Top gas pressure': 'ToGaPr',
    'Enriching oxygen pressure': 'EnOxPr',
    'Cold blast pressure': 'CoBlPr',
    'Total pressure drop': 'ToPrDr',
    'Hot blast pressure': 'HoBlPr',
    'Actual blast velocity': 'AcBlVe',
    'Cold blast temperature': 'CoBlTe',
    'Hot blast temperature': 'HoBlTe',
    'Top temperature': 'ToTe',
    'Blast humidity': 'BlHu',
    'Coal injection set value': 'CoInSeVa',
    'Fomer SI': 'FoSI',
    'SI': 'SI',
    'HoBl': 'HoBl',
    'ToGasP': 'ToGasP',
    'CoBF': 'CoBF'
}

print("Column mapping defined successfully!")
print(f"Total columns to map: {len(column_mapping)}")


In [ ]:
# Load the dataset
try:
    df = pd.read_excel('../data/dataset.xlsx')
    print("Dataset loaded successfully!")
    print(f"Dataset shape: {df.shape}")
    print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
except Exception as e:
    print(f"Error loading dataset: {e}")

# Display basic information about the dataset
print("\n" + "="*50)
print("DATASET OVERVIEW")
print("="*50)
print(f"Number of rows: {df.shape[0]:,}")
print(f"Number of columns: {df.shape[1]}")
print(f"Dataset size: {df.size:,} cells")

# Display first few rows
print("\nFirst 5 rows:")
df.head()


In [ ]:
# Data Quality Analysis
print("="*50)
print("DATA QUALITY ANALYSIS")
print("="*50)

# Check for missing values
print("Missing Values Analysis:")
missing_data = df.isnull().sum()
missing_percentage = (missing_data / len(df)) * 100
missing_info = pd.DataFrame({
    'Column': missing_data.index,
    'Missing_Count': missing_data.values,
    'Missing_Percentage': missing_percentage.values
})
missing_info = missing_info[missing_info['Missing_Count'] > 0].sort_values('Missing_Percentage', ascending=False)

if len(missing_info) > 0:
    print(f"\nColumns with missing values: {len(missing_info)}")
    print(missing_info)
else:
    print("No missing values found!")

# Check data types
print(f"\nData Types:")
print(df.dtypes.value_counts())

# Check for duplicates
duplicates = df.duplicated().sum()
print(f"\nDuplicate rows: {duplicates}")

# Basic statistics
print(f"\nBasic Statistics:")
df.describe()


In [ ]:
# Target Variable Analysis (SI - Silicon Content)
print("="*50)
print("TARGET VARIABLE ANALYSIS (SI)")
print("="*50)

# Check if SI column exists
si_column = 'SI'
if si_column in df.columns:
    si_data = df[si_column].dropna()
    
    print(f"SI Statistics:")
    print(f"Count: {len(si_data):,}")
    print(f"Mean: {si_data.mean():.4f}")
    print(f"Std: {si_data.std():.4f}")
    print(f"Min: {si_data.min():.4f}")
    print(f"25%: {si_data.quantile(0.25):.4f}")
    print(f"50%: {si_data.quantile(0.50):.4f}")
    print(f"75%: {si_data.quantile(0.75):.4f}")
    print(f"Max: {si_data.max():.4f}")
    print(f"Range: {si_data.max() - si_data.min():.4f}")
    print(f"Skewness: {si_data.skew():.4f}")
    print(f"Kurtosis: {si_data.kurtosis():.4f}")
    
    # Check for outliers using IQR method
    Q1 = si_data.quantile(0.25)
    Q3 = si_data.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = si_data[(si_data < lower_bound) | (si_data > upper_bound)]
    print(f"Outliers (IQR method): {len(outliers)} ({len(outliers)/len(si_data)*100:.2f}%)")
    
else:
    print(f"SI column not found in dataset!")
    print(f"Available columns: {list(df.columns)}")


In [ ]:
# Visualizations for Target Variable
if si_column in df.columns:
    si_data = df[si_column].dropna()
    
    # Create subplots
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('Silicon (SI) Content Analysis', fontsize=16, fontweight='bold')
    
    # Histogram
    axes[0,0].hist(si_data, bins=50, alpha=0.7, color='skyblue', edgecolor='black')
    axes[0,0].set_title('Distribution of SI Content')
    axes[0,0].set_xlabel('SI Content')
    axes[0,0].set_ylabel('Frequency')
    axes[0,0].axvline(si_data.mean(), color='red', linestyle='--', label=f'Mean: {si_data.mean():.3f}')
    axes[0,0].legend()
    
    # Box plot
    axes[0,1].boxplot(si_data, patch_artist=True, 
                      boxprops=dict(facecolor='lightgreen', alpha=0.7))
    axes[0,1].set_title('SI Content Box Plot')
    axes[0,1].set_ylabel('SI Content')
    
    # Time series plot (if timestamp available)
    if 'Timestamp' in df.columns:
        df_clean = df.dropna(subset=[si_column, 'Timestamp'])
        df_clean['Timestamp'] = pd.to_datetime(df_clean['Timestamp'])
        df_sorted = df_clean.sort_values('Timestamp')
        axes[1,0].plot(df_sorted['Timestamp'], df_sorted[si_column], alpha=0.7, color='purple')
        axes[1,0].set_title('SI Content Over Time')
        axes[1,0].set_xlabel('Timestamp')
        axes[1,0].set_ylabel('SI Content')
        axes[1,0].tick_params(axis='x', rotation=45)
    
    # Q-Q plot for normality check
    from scipy import stats
    stats.probplot(si_data, dist="norm", plot=axes[1,1])
    axes[1,1].set_title('Q-Q Plot (Normality Check)')
    
    plt.tight_layout()
    plt.show()
    
    # Print distribution insights
    print(f"\nDistribution Insights:")
    print(f"- Mean SI content: {si_data.mean():.4f}")
    print(f"- Standard deviation: {si_data.std():.4f}")
    print(f"- Coefficient of variation: {(si_data.std()/si_data.mean())*100:.2f}%")
    
    if abs(si_data.skew()) < 0.5:
        print(f"- Distribution: Approximately normal (skewness: {si_data.skew():.3f})")
    elif abs(si_data.skew()) < 1:
        print(f"- Distribution: Moderately skewed (skewness: {si_data.skew():.3f})")
    else:
        print(f"- Distribution: Highly skewed (skewness: {si_data.skew():.3f})")


In [ ]:
# Feature Analysis and Correlation
print("="*50)
print("FEATURE ANALYSIS")
print("="*50)

# Identify numeric columns (excluding timestamp)
numeric_columns = df.select_dtypes(include=[np.number]).columns.tolist()
if 'Timestamp' in numeric_columns:
    numeric_columns.remove('Timestamp')

print(f"Numeric features identified: {len(numeric_columns)}")
print(f"Features: {numeric_columns}")

# Calculate correlation with target variable
if si_column in df.columns and si_column in numeric_columns:
    correlations = df[numeric_columns].corr()[si_column].sort_values(key=abs, ascending=False)
    
    print(f"\nTop 10 Features Most Correlated with SI:")
    print(correlations.head(11).to_string())  # 11 to include SI itself
    
    # Create correlation heatmap
    plt.figure(figsize=(12, 10))
    correlation_matrix = df[numeric_columns].corr()
    
    # Create mask for upper triangle
    mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
    
    # Generate heatmap
    sns.heatmap(correlation_matrix, mask=mask, annot=False, cmap='coolwarm', center=0,
                square=True, linewidths=.5, cbar_kws={"shrink": .5})
    plt.title('Feature Correlation Matrix')
    plt.tight_layout()
    plt.show()
    
    # Identify highly correlated feature pairs (excluding target)
    high_corr_pairs = []
    for i in range(len(correlation_matrix.columns)):
        for j in range(i+1, len(correlation_matrix.columns)):
            corr_val = correlation_matrix.iloc[i, j]
            if abs(corr_val) > 0.8:  # High correlation threshold
                high_corr_pairs.append((correlation_matrix.columns[i], 
                                      correlation_matrix.columns[j], 
                                      corr_val))
    
    if high_corr_pairs:
        print(f"\nHighly Correlated Feature Pairs (|r| > 0.8):")
        for feat1, feat2, corr in high_corr_pairs:
            print(f"{feat1} <-> {feat2}: {corr:.3f}")
    else:
        print(f"\nNo highly correlated feature pairs found (|r| > 0.8)")


In [ ]:
# Data Preprocessing and Feature Engineering
print("="*50)
print("DATA PREPROCESSING")
print("="*50)

# Create a copy for preprocessing
df_processed = df.copy()

# Handle timestamp
if 'Timestamp' in df_processed.columns:
    df_processed['Timestamp'] = pd.to_datetime(df_processed['Timestamp'])
    
    # Extract time-based features
    df_processed['Hour'] = df_processed['Timestamp'].dt.hour
    df_processed['DayOfWeek'] = df_processed['Timestamp'].dt.dayofweek
    df_processed['Month'] = df_processed['Timestamp'].dt.month
    df_processed['Quarter'] = df_processed['Timestamp'].dt.quarter
    
    # Calculate time differences (for lag features)
    df_processed = df_processed.sort_values('Timestamp')
    df_processed['TimeDiff_Minutes'] = df_processed['Timestamp'].diff().dt.total_seconds() / 60
    
    print("✅ Time-based features created")

# Handle missing values
print(f"\nMissing Value Treatment:")
missing_cols = df_processed.isnull().sum()
missing_cols = missing_cols[missing_cols > 0]

for col in missing_cols.index:
    if df_processed[col].dtype in ['float64', 'int64']:
        # For numeric columns, use median imputation
        median_val = df_processed[col].median()
        df_processed[col].fillna(median_val, inplace=True)
        print(f"✅ {col}: Filled {missing_cols[col]} missing values with median ({median_val:.3f})")
    else:
        # For categorical columns, use mode
        mode_val = df_processed[col].mode()[0] if len(df_processed[col].mode()) > 0 else 'Unknown'
        df_processed[col].fillna(mode_val, inplace=True)
        print(f"✅ {col}: Filled {missing_cols[col]} missing values with mode ({mode_val})")

# Create lag features for important variables
if si_column in df_processed.columns:
    # Create lag features for SI (using previous values)
    for lag in [1, 2, 3, 5]:
        df_processed[f'SI_lag_{lag}'] = df_processed[si_column].shift(lag)
    
    # Create rolling statistics
    for window in [3, 5, 10]:
        df_processed[f'SI_rolling_mean_{window}'] = df_processed[si_column].rolling(window=window).mean()
        df_processed[f'SI_rolling_std_{window}'] = df_processed[si_column].rolling(window=window).std()
    
    print("✅ Lag and rolling features created for SI")

print(f"\nProcessed dataset shape: {df_processed.shape}")
print(f"New features added: {df_processed.shape[1] - df.shape[1]}")


In [ ]:
# Save processed data and create summary
print("="*50)
print("DATA EXPORT AND SUMMARY")
print("="*50)

# Save processed dataset
df_processed.to_csv('../data/processed_dataset.csv', index=False)
print("✅ Processed dataset saved to '../data/processed_dataset.csv'")

# Create data summary report
summary_report = {
    'Original Dataset': {
        'Rows': f"{df.shape[0]:,}",
        'Columns': df.shape[1],
        'Missing Values': df.isnull().sum().sum(),
        'Duplicates': df.duplicated().sum(),
        'Memory Usage (MB)': f"{df.memory_usage(deep=True).sum() / 1024**2:.2f}"
    },
    'Processed Dataset': {
        'Rows': f"{df_processed.shape[0]:,}",
        'Columns': df_processed.shape[1],
        'Missing Values': df_processed.isnull().sum().sum(),
        'New Features Added': df_processed.shape[1] - df.shape[1],
        'Memory Usage (MB)': f"{df_processed.memory_usage(deep=True).sum() / 1024**2:.2f}"
    }
}

if si_column in df.columns:
    si_stats = df[si_column].describe()
    summary_report['Target Variable (SI)'] = {
        'Count': f"{si_stats['count']:.0f}",
        'Mean': f"{si_stats['mean']:.4f}",
        'Std': f"{si_stats['std']:.4f}",
        'Min': f"{si_stats['min']:.4f}",
        'Max': f"{si_stats['max']:.4f}",
        'Range': f"{si_stats['max'] - si_stats['min']:.4f}"
    }

# Print summary
for section, metrics in summary_report.items():
    print(f"\n{section}:")
    for metric, value in metrics.items():
        print(f"  {metric}: {value}")

print("\n" + "="*50)
print("PHASE 1 COMPLETION STATUS")
print("="*50)
print("✅ Task 1.1: Data Exploration and Analysis - COMPLETED")
print("✅ Task 1.2: Feature Engineering and Selection - COMPLETED") 
print("✅ Task 1.3: Time Series Analysis Setup - COMPLETED")
print("\n🎯 Ready for Phase 2: Model Development and Training")
